In [232]:
# pip install dash-bootstrap-components

In [233]:
import pandas as pd, plotly.express as px
import dash_bootstrap_components as dbc
from dash import dcc, html, Dash, Input, Output

In [234]:

df = pd.read_csv('consdata.csv')  #Reading dataset
df['Date Time Served'] = pd.to_datetime(df['Date Time Served'])
df['Opening Balance (ml)'] = pd.to_numeric(df['Opening Balance (ml)'])
df['Purchase (ml)'] = pd.to_numeric(df['Purchase (ml)'])
df['Consumed (ml)'] = pd.to_numeric(df['Consumed (ml)'])
df['Closing Balance (ml)'] = pd.to_numeric(df['Closing Balance (ml)'])
df['Day'] = df['Date Time Served'].dt.day
df['Month'] = df['Date Time Served'].dt.month
df['Year'] = df['Date Time Served'].dt.year
df['Time (Hour)'] = df['Date Time Served'].dt.hour
df['Time (Mins)'] = df['Date Time Served'].dt.minute
dfCopy = df.sort_values(['Year', 'Month', 'Day', 'Time (Hour)', 'Time (Mins)'], ascending=True).copy()
dfCopy

df2023 = dfCopy[dfCopy['Year'].astype('str').str.contains('2023')].copy()
df2023

# dfNone = dfCopy[dfCopy['Bar Name'].str.contains('None')].copy()
# dfNone


,Date Time Served,Bar Name,Alcohol Type,Brand Name,Opening Balance (ml),Purchase (ml),Consumed (ml),Closing Balance (ml),Day,Month,Year,Time (Hour),Time (Mins)
1,2023-01-01 10:07:00,Smith's Bar,Wine,Yellow Tail,1344.37,0.0,0.00,1344.37,1,1,2023,10,7
5,2023-01-01 10:28:00,Brown's Bar,Vodka,Grey Goose,2560.44,0.0,0.00,2560.44,1,1,2023,10,28
2,2023-01-01 11:26:00,Johnson's Bar,Vodka,Grey Goose,1034.28,0.0,0.00,1034.28,1,1,2023,11,26
13,2023-01-01 11:32:00,Anderson's Bar,Whiskey,Jim Beam,872.08,0.0,0.00,872.08,1,1,2023,11,32
3,2023-01-01 13:53:00,Johnson's Bar,Beer,Coors,2194.53,0.0,0.00,2194.53,1,1,2023,13,53
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6551,2023-12-31 19:29:00,Taylor's Bar,Whiskey,Jim Beam,859.78,0.0,349.75,510.03,31,12,2023,19,29
6550,2023-12-31 19:58:00,Taylor's Bar,Rum,Malibu,2026.72,0.0,220.20,1806.52,31,12,2023,19,58
6553,2023-12-31 20:19:00,Anderson's Bar,Wine,Yellow Tail,758.06,0.0,129.41,628.65,31,12,2023,20,19
6548,2023-12-31 20:47:00,Taylor's Bar,Wine,Sutter Home,0.00,0.0,0.00,0.00,31,12,2023,20,47


In [235]:
app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])

In [236]:
app.layout = ([

        dbc.Row([
            dbc.Col([
                html.H1('Figures for the following dataset!',className='text-center my-5'),
                html.Div('Dash _AsadSDASD', className='text-right my-15')
            ])
        ]),
        dbc.Row([
            dbc.Col([
                dbc.Card([
                    dbc.CardBody([
                        html.H3('This is a graph of how the bars are related to the month of that year', className='card-title'),
                        dcc.Dropdown(id = 'monthSelected',
                                     options=[{'label':month, 'value':month} for month in df2023['Month'].unique()],
                                     value=None,
                                     placeholder='Select a month'),
                        dcc.Graph(id = 'barToMonth')

                    ])
                ])
            ])
        ])
])
@app.callback(
    Output('barToMonth', 'figure'),
    Input('monthSelected', 'value')
)
def update_month(monthOfTheYear):
    if monthOfTheYear:
        values_to_remove = [0]
        df2023.drop(index=('Purchase (ml)', 0))
        filteredDf = df2023[df2023['Month'] == monthOfTheYear]
    else:
        filteredDf = df2023

    figBarYearToMonth = px.histogram(filteredDf, x='Purchase (ml)', color='Bar Name', barmode='group', color_discrete_sequence=px.colors.qualitative.G10, template='plotly_dark', title='Each month to all bars to Purchase(ml)')

    return figBarYearToMonth


In [237]:
if __name__ == '__main__':
    app.run(port=4050)